In [19]:
import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

3375

In [20]:
from openai import OpenAI
model_name = "medgemma-4b-it"
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
)

In [40]:
import textwrap
import base64
import requests
import mimetypes
import PIL.Image
from IPython.display import Markdown, Image

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def convert_to_base64(image_path):
    if image_path.startswith("http"):
        response = requests.get(image_path)
        if response.status_code != 200:
            raise Exception(f"Failed to download image: {response.status_code}")
        mime_type = response.headers.get("Content-Type", "image/jpeg")
        encoded = base64.b64encode(response.content).decode("utf-8")
    else:
        mime_type, _ = mimetypes.guess_type(image_path)
        if mime_type is None:
            mime_type = "image/jpeg"
        with open(image_path, "rb") as f:
            encoded = base64.b64encode(f.read()).decode("utf-8")
    
    return f"data:{mime_type};base64,{encoded}"

In [44]:
def call_LMM(image_path:str, prompt:str) -> str:
    # img = PIL.image.open(image_path)
    response = client.chat.completions.create(
        model=model_name,
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": convert_to_base64(image_path),
                    },
                },
            ],
        }],
    )

    return to_markdown(response.choices[0].message.content)

In [45]:
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
prompt = "What's in this image?"
call_LMM(image_path, prompt)

> The image shows a wide, open field of tall green grass. A wooden walkway or bridge stretches across the field, leading into the distance. The sky is bright blue with some scattered clouds. There are trees and vegetation visible on either side of the path. It appears to be a landscape photograph taken from a high vantage point.
